### keras-retinanet 패키지를 이용하여 이미지와 영상 Object Detection 수행
*  Pretrained된 coco 모델을 로드 하고 이를 이용하여 Object Detection 수행

### 본 실습 예제는 GPU를 활용하므로 상단 메뉴에서 런타임->런타임 유형 변경에서 GPU를 선택해 주십시요.

### tensorflow, keras 설치 및 강의 실습코드/데이터 Download

Colab 버전은 colab에서 pip 명령어를 이용하여 tensorflow 1.15, keras 2.3를 소스코드 커널 기동시 마다 설치해야 합니다

Colab의 tensorflow는 2020년 기준으로 2.2 이며, keras는 2.3입니다. 실습코드는 tensorflow 1.15 기준으로 되어 있으므로 이를 downgrade해야 합니다.

pip를 이용하여 tensorflow 1.15을 설치하면 자동으로 downgrade 됩니다.

OpenCV는 Colab에서 이미 설치 되어 있으니 추가설치는 필요 없습니다.

강의 실습코드와 데이터는 https://github.com/chulminkw/DLCV.git 에서 다운로드 할 수 있습니다. 

In [1]:
# 현재 디렉토리는 /content이며 이 디렉토리를 기준으로 실습코드와 데이터를 다운로드 합니다. 
!pwd
!rm -rf DLCV
!git clone https://github.com/chulminkw/DLCV.git
# DLCV 디렉토리가 Download되고 DLCV 밑에 Detection과 Segmentation 디렉토리가 있는 것을 확인
!ls -lia 
!ls -lia DLCV

# tensorflow 1.15을 설치합니다. 자동으로 tensorflow 2.2가 1.15으로 downgrade 됩니다. 
!pip install tensorflow-gpu==1.15.2 
# keras 2.3를 설치합니다. 
!pip install keras==2.3.0

/content
Cloning into 'DLCV'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 145 (delta 17), reused 0 (delta 0), pack-reused 89
Receiving objects: 100% (145/145), 127.44 MiB | 33.43 MiB/s, done.
Resolving deltas: 100% (38/38), done.
total 20
4194328 drwxr-xr-x 1 root root 4096 Jul 31 10:04 .
4208240 drwxr-xr-x 1 root root 4096 Jul 31 10:00 ..
4194329 drwxr-xr-x 1 root root 4096 Jul 28 16:16 .config
4208329 drwxr-xr-x 6 root root 4096 Jul 31 10:04 DLCV
1310744 drwxr-xr-x 1 root root 4096 Jul 10 16:29 sample_data
total 160
4208329 drwxr-xr-x 6 root root   4096 Jul 31 10:04 .
4194328 drwxr-xr-x 1 root root   4096 Jul 31 10:04 ..
4208406 drwxr-xr-x 6 root root   4096 Jul 31 10:04 data
4208372 drwxr-xr-x 8 root root   4096 Jul 31 10:04 Detection
4208357 -rw-r--r-- 1 root root 134412 Jul 31 10:04 DLCV_Colab_SrcCode.zip
4208330 drwxr-xr-x 8 root root   4096 Jul 31 10:04 .git
4208398 -rw-r--r

In [2]:
# tensorflow는 1.15, keras는 2.3 버전 확인
import tensorflow as tf
import keras

print(tf.__version__)
print(keras.__version__)

# gpu가 세팅되어 있는지 확인. 
tf.test.gpu_device_name()

Using TensorFlow backend.


1.15.2
2.3.0


'/device:GPU:0'

#### fizyr keras-retinanet package를 설치합니다. 
/content/DLCV/Detection/retina/keras-retinanet 디렉토리에 download 되고 설치 됩니다. 


In [3]:
%cd /content/DLCV/Detection/retina
!git clone https://github.com/fizyr/keras-retinanet.git

/content/DLCV/Detection/retina
Cloning into 'keras-retinanet'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 5893 (delta 3), reused 0 (delta 0), pack-reused 5885
Receiving objects: 100% (5893/5893), 13.41 MiB | 23.68 MiB/s, done.
Resolving deltas: 100% (3967/3967), done.


In [4]:
%cd /content/DLCV/Detection/retina/keras-retinanet
!pip install . --user 
!python setup.py build_ext --inplace

/content/DLCV/Detection/retina/keras-retinanet
Processing /content/DLCV/Detection/retina/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=170841 sha256=d83fdde4519bc8b8933f57bfdc3af48ce1de809d058522475b88f26fde322964
  Stored in directory: /root/.cache/pip/wheels/78/11/a0/e7d32b794790f97776b6d352fbb95de0eb246ebbdb5515c99a
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13346 sha256=1d1a7526c3ad1d4bd761d9b005a7994b323dca84213536aaa88982a7d7c6dfb7
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Mai

In [1]:
# 아래 import로 keras-retinanet이 정상적으로 설치되어 있는지 확인. 특히 backbone 함수 체크. 
import tensorflow as tf

from keras_retinanet import models
from keras_retinanet.models import backbone

b = backbone('resnet50')

Using TensorFlow backend.


#### 관련 모듈 import 

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
%matplotlib inline


# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

In [ ]:
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

#### gpu 환경 설정

In [ ]:
# gpu id 값 설정
gpu = 0
setup_gpu(gpu)

#### keras-retinanet으로 pretrained된 coco 모델 다운로드하고 해당 모델을 로드
* 코렙 버전은 /content/DLCV/Detection/retina/keras-retinanet/snapshots 디렉토리 밑에 pretrained 모델을 download

In [ ]:
# 아래 모델은 https://github.com/fizyr/keras-retinanet/releases 에서 download 받을 수 있음. 
# 해당 모델 h5 파일을 snapshot 디렉토리에 저장 후 retina model의 load_model()을 이용하여 모델 로딩.
%cd  /content/DLCV/Detection/retina/keras-retinanet/snapshots
!wget https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5 

In [ ]:
# 코렙 버전 디렉토리 수정. 
os.path.join('/content/DLCV/Detection/retina/keras-retinanet', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')

In [ ]:
!ls /content/DLCV/Detection/retina/keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5

In [ ]:
model_path = os.path.join('/content/DLCV/Detection/retina/keras-retinanet', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')
print(model_path)
# pretrained coco 모델 파일을 retinanet 모델로 로딩.  
from keras_retinanet import models
retina_model = models.load_model(model_path, backbone_name='resnet50')

#### coco 데이터 세트의 클래스id별 클래스명 지정. 

In [ ]:
labels_to_names_seq = {0:'person',1:'bicycle',2:'car',3:'motorbike',4:'aeroplane',5:'bus',6:'train',7:'truck',8:'boat',9:'traffic light',10:'fire hydrant',
                        11:'stop sign',12:'parking meter',13:'bench',14:'bird',15:'cat',16:'dog',17:'horse',18:'sheep',19:'cow',20:'elephant',
                        21:'bear',22:'zebra',23:'giraffe',24:'backpack',25:'umbrella',26:'handbag',27:'tie',28:'suitcase',29:'frisbee',30:'skis',
                        31:'snowboard',32:'sports ball',33:'kite',34:'baseball bat',35:'baseball glove',36:'skateboard',37:'surfboard',38:'tennis racket',39:'bottle',40:'wine glass',
                        41:'cup',42:'fork',43:'knife',44:'spoon',45:'bowl',46:'banana',47:'apple',48:'sandwich',49:'orange',50:'broccoli',
                        51:'carrot',52:'hot dog',53:'pizza',54:'donut',55:'cake',56:'chair',57:'sofa',58:'pottedplant',59:'bed',60:'diningtable',
                        61:'toilet',62:'tvmonitor',63:'laptop',64:'mouse',65:'remote',66:'keyboard',67:'cell phone',68:'microwave',69:'oven',70:'toaster',
                        71:'sink',72:'refrigerator',73:'book',74:'clock',75:'vase',76:'scissors',77:'teddy bear',78:'hair drier',79:'toothbrush' }

In [ ]:
test_array = np.array([[1,2]])
print(test_array.shape)
expand_array = np.expand_dims(test_array, axis=0)
expand_array.shape

#### 원본 이미지를 Object Detection 후에 시각화 표시. 

In [ ]:
# load image, 코랩 버전 절대 경로 변경
default_dir = '/content/DLCV'
image = read_image_bgr(os.path.join(default_dir, 'data/image/beatles01.jpg'))
print('image shape:', image.shape)
# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)


# 모델에 입력전에 이미지 사전 처리. keras-retinanet은 image
image = preprocess_image(image)
image, scale = resize_image(image)
print('resized image size:', image.shape, 'scale:', scale)

# 이미지에 대해 Object Detection 수행. 
start = time.time()
boxes, scores, labels = retina_model.predict_on_batch(np.expand_dims(image, axis=0))
print(boxes.shape, scores.shape, labels.shape)
print("processing time: ", time.time() - start)

# correct for image scale
boxes /= scale

# visualize detections
for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores are sorted so we can break
    if score < 0.5:
        break
        
    color = label_color(label)
    
    b = box.astype(int)
    draw_box(draw, b, color=color)
    
    caption = "{} {:.3f}".format(labels_to_names_seq[label], score)
    draw_caption(draw, b, caption)
    
plt.figure(figsize=(14, 14))
plt.axis('off')
plt.imshow(draw)
plt.show()

In [ ]:
def get_detected_image_retina(model, img_array, use_copied_array, is_print=True):
    
    # copy to draw on
    draw_img = None
    if use_copied_array:
        draw_img = img_array.copy()
    else:
        draw_img = img_array
    
    img_array = preprocess_image(img_array)
    img_array, scale = resize_image(img_array)
    
    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(img_array, axis=0))
    if is_print:
        print("object detection 처리 시간: ", round(time.time() - start,5))
    
    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw_img, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names_seq[label], score)
        draw_caption(draw_img, b, caption)
    
    if is_print:
        print("이미지 processing 시간: ", round(time.time() - start,5))
    
    return draw_img

# image 로드 
default_dir = '/content/DLCV'
img = cv2.imread(os.path.join(default_dir, 'data/image/john_wick01.jpg'))
draw_img = get_detected_image_retina(retina_model, img, use_copied_array=True, is_print=True)
img_rgb = cv2.cvtColor(draw_img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(14, 14))
plt.imshow(img_rgb)

In [ ]:
def detect_video_retina(model, input_path, output_path=""):
    
    start = time.time()
    cap = cv2.VideoCapture(input_path)
    
    codec = cv2.VideoWriter_fourcc(*'XVID')
    vid_fps = cap.get(cv2.CAP_PROP_FPS)
    vid_size= (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    vid_writer = cv2.VideoWriter(output_path, codec, vid_fps, vid_size)
    
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('총 Frame 갯수:', frame_cnt)
    
    while True:
        hasFrame, image_frame = cap.read()
        if not hasFrame:
            print('프레임이 없거나 종료 되었습니다.')
            break

        detected_image = get_detected_image_retina(model,image_frame, use_copied_array=False, is_print=True)
        vid_writer.write(detected_image)
    
    vid_writer.release()
    cap.release()
    print('### Video Detect 총 수행시간:', round(time.time()-start, 5))

In [ ]:
default_dir = '/content/DLCV'
detect_video_retina(retina_model, os.path.join(default_dir, 'data/video/John_Wick_small.mp4'), 
                    output_path=os.path.join(default_dir, "data/output/John_Wick_small_retina01.avi"))

In [ ]:
# 코랩 버전은 google drive 이용. 
!gsutil cp ../../data/output/John_Wick_small_retina01.avi gs://my_bucket_dlcv/data/output/John_Wick_small_retina01.avi

In [ ]:
default_dir = '/content/DLCV'
detect_video_retina(retina_model, os.path.join(default_dir, 'data/video/Night_Day_Chase.mp4'), 
                    output_path=os.path.join(default_dir, "data/output/Night_Day_Chase_retina01.avi"))

In [ ]:
!gsutil cp ../../data/output/Night_Day_Chase_retina01.avi gs://my_bucket_dlcv/data/output/Night_Day_Chase_retina01.avi